In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import firebase_admin
from firebase_admin import credentials, firestore, auth
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from joblib import load
from flask import Flask, request, jsonify

In [3]:
cred = credentials.Certificate('./firebase-sdk.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

In [4]:
feature_order = ['Name', 'Gender', 'Age', 'CountryofBirth', 'Smoking', 'Alcohol', 'FamilyHistory', 'CloseContact', 'HealthConditions', 'OccupationalRisks', 'PhysicalActivity', 'Diet', 'AirPollution', 'LongTermCough', 'ChestPain', 'AppetiteLoss', 'WeightLoss', 'Chills', 'Fatigue', 'NightSweat', 'CoughBlood', 'Fevers', 'NitricOxide']

In [7]:
# Function to fetch a user's data from Firestore
def fetch_user_data(user_id):
    doc_ref = db.collection('users').document(user_id)
    doc = doc_ref.get()
    if doc.exists:
        print(doc.to_dict())
        return doc.to_dict()
    else:
        print("No such document!")
        return None

# Function to preprocess user's data
def preprocess_user_data(user_data):
    # Assuming user_data is a dictionary containing the user's features
    df = pd.DataFrame([user_data])  # Convert dict to DataFrame
    df = df[feature_order]
    # Apply the same preprocessing as your training data
    # Load label encoder and transform 'Country of Birth'
    le_country = load('label_encoder.joblib')
    df['encoded_country'] = le_country.transform(df['CountryofBirth'])
    
    # Ensure all preprocessing steps match training, including dropping unused columns
    processed_df = df.drop(['Name', 'CountryofBirth'], axis=1)
    
    # Return preprocessed user data ready for prediction
    return processed_df

# Load your trained Decision Tree model
classifier = load('classifier.joblib')

# Predict function for a user
def predict_tbpred(user_id):
    user_data = fetch_user_data(user_id)
    if user_data:
        processed_data = preprocess_user_data(user_data)
        prediction = classifier.predict(processed_data)
        return prediction
    else:
        return "User data not found"


# Example usage
user_id = '4PzkRocqvKdL71AtKXiu'
prediction = predict_tbpred(user_id)
print(f"Predicted tb_pred for user {user_id}: {prediction}")

{'Fatigue': '1', 'NightSweat': '1', 'CountryofBirth': 'Samoa', 'HealthConditions': '1', 'LongTermCough': '1', 'CoughBlood': '1', 'AirPollution': '3', 'Diet': '1', 'Fevers': '1', 'Alcohol': '2', 'Name': 'sadsa', 'PhysicalActivity': '1', 'Chills': '1', 'WeightLoss': '1', 'AppetiteLoss': '1', 'NitricOxide': 20, 'OccupationalRisks': '1', 'Gender': '1', 'CloseContact': '1', 'ChestPain': '1', 'FamilyHistory': '1', 'Age': '123', 'Smoking': '2'}
Predicted tb_pred for user 4PzkRocqvKdL71AtKXiu: [1]


In [19]:
import numpy as np
from joblib import load
import pandas as pd

df = pd.read_csv("./dataset/tbpred.csv")

# Get the feature names from the column headers
feature_names = df.columns.tolist()
print(feature_names)

classifier = load('classifier.joblib')
a = np.array([[1,24,0,4,1,0,0,1,1,1,1,3,0,0,1,1,0,1,0,0,0,34.8]])
print(classifier.predict(a))

['Name', 'Gender', 'Age', 'CountryofBirth', 'Smoking', 'Alcohol', 'FamilyHistory', 'CloseContact', 'HealthConditions', 'OccupationalRisks', 'PhysicalActivity', 'Diet', 'AirPollution', 'LongTermCough', 'ChestPain', 'AppetiteLoss', 'WeightLoss', 'Chills', 'Fatigue', 'NightSweat', 'CoughBlood', 'Fevers', 'NitricOxide', 'tb_pred']
[0]


C:\Users\alice\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"
